In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from torch.utils.data import TensorDataset, Dataset, DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from matplotlib import pyplot as plt
import numpy as np
import pathlib
import shutil
import tempfile
import pandas as pd
import seaborn as sns
import copy
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, TargetEncoder, OrdinalEncoder

In [3]:
torch.cuda.is_available()

True

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

In [58]:
class TinyModel(nn.Module):
    def __init__(self, input_size):
        super(TinyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 1)
        self.activation = nn.ELU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

class SmallModel(nn.Module):
    def __init__(self, input_size):
        super(SmallModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)
        self.activation = nn.ELU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

class MediumModel(nn.Module):
    def __init__(self, input_size):
        super(MediumModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 1)
        self.activation = nn.ELU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

class LargeModel(nn.Module):
    def __init__(self, input_size):
        super(LargeModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 1)
        self.activation = nn.ELU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        x = self.fc5(x)
        return x

class DropoutModel(nn.Module):
    def __init__(self, input_size):
        super(DropoutModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 1)
        self.activation = nn.ELU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.activation(self.fc3(x))
        x = self.dropout(x)
        x = self.activation(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

In [9]:
batch_size = 512  # Adjust batch size according to your needs

In [10]:
pd.options.mode.chained_assignment = None  # default='warn'

In [11]:
df = pd.read_csv('adult.csv')
df = df.replace([np.inf, -np.inf], np.nan)
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
nan_counts = df.isna().sum()
nan_counts

In [ ]:
df.income.value_counts()

In [ ]:
target_variable = 'income'

# Identify categorical features (excluding the target if it's included as categorical)
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove(target_variable)  # Remove the target variable from the list if it's categorical

# Identify numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
# Create and print contingency tables for each categorical feature
#for feature in categorical_features:
#    contingency_table = pd.crosstab(df[feature], df[target_variable])
#    display(contingency_table)

In [ ]:
#sns.pairplot(data=df, hue="income")

In [ ]:
# Plot violin plots for each numerical feature
#for feature in numerical_features:
#    sns.violinplot(x='income', y=feature, data=df)
#    plt.title(f'Violin plot of {feature}')
#    plt.show()

# Preprocessing

In [12]:
target_variable = 'income'

In [13]:
# Split the dataset into training and testing sets
X = df.drop(target_variable, axis=1)
y = ( df[target_variable] == ">50K").astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Identify categorical features
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove(target_variable)
# Identify numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create the preprocessing pipelines for both numerical and categorical data
numerical_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('encoder', TargetEncoder())
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

In [25]:
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [26]:
X_train_preprocessed.shape

(39073, 14)

In [27]:
X_test_torch = torch.tensor(X_test_preprocessed, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32)

test_dataset = TensorDataset(X_test_torch, y_test_torch)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=8, persistent_workers=True)

# Numero di campioni e performance

In [46]:
dtModels = {}

rf_model = RandomForestClassifier(random_state=42)
dtModels["RandomForest"] = rf_model

dt_model = DecisionTreeClassifier(random_state=42)
dtModels["DecisionTree"] = dt_model

xgb_model = xgb.XGBClassifier(random_state=42)
dtModels["XGBoost"] = xgb_model

lgb_model = lgb.LGBMClassifier(random_state=42, verbose=-1)
dtModels["LGBM"] = lgb_model

cat_model = CatBoostClassifier(random_state=42, verbose=False)
dtModels["CatBoost"] = cat_model

In [47]:
# Usage:
input_size = X_train_preprocessed.shape[1]  # Assuming FEATURES is defined elsewhere in your code
tiny_model = TinyModel(input_size)
small_model = SmallModel(input_size)
medium_model = MediumModel(input_size)
large_model = LargeModel(input_size)
dropout_model = DropoutModel(input_size)

neuralNetsModels = {
    "tiny_model": NeuralNet(tiny_model),
    "small_model": NeuralNet(small_model),
    "medium_model": NeuralNet(medium_model),
    "large_model": NeuralNet(large_model),
    "dropout_model": NeuralNet(dropout_model)
}

In [48]:
size_histories = {}

In [49]:
sizes = [int(len(X_train)/100), int(len(X_train)/10), len(X_train)]

In [50]:
columns = pd.MultiIndex.from_product([sizes, ["Train", "Test"]], names = ["size", "split"])
list_embedding = [name + "_emb" for name in neuralNetsModels.keys()]
results = pd.DataFrame(columns = columns, index = list(dtModels.keys()) + list(neuralNetsModels.keys()) + list_embedding)
results

size              390        3907       39073     
split             Train Test Train Test Train Test
RandomForest        NaN  NaN   NaN  NaN   NaN  NaN
DecisionTree        NaN  NaN   NaN  NaN   NaN  NaN
XGBoost             NaN  NaN   NaN  NaN   NaN  NaN
LGBM                NaN  NaN   NaN  NaN   NaN  NaN
CatBoost            NaN  NaN   NaN  NaN   NaN  NaN
tiny_model          NaN  NaN   NaN  NaN   NaN  NaN
small_model         NaN  NaN   NaN  NaN   NaN  NaN
medium_model        NaN  NaN   NaN  NaN   NaN  NaN
large_model         NaN  NaN   NaN  NaN   NaN  NaN
dropout_model       NaN  NaN   NaN  NaN   NaN  NaN
tiny_model_emb      NaN  NaN   NaN  NaN   NaN  NaN
small_model_emb     NaN  NaN   NaN  NaN   NaN  NaN
medium_model_emb    NaN  NaN   NaN  NaN   NaN  NaN
large_model_emb     NaN  NaN   NaN  NaN   NaN  NaN
dropout_model_emb   NaN  NaN   NaN  NaN   NaN  NaN

# Defining the Models
## Tree Based Models

In [28]:
param_grid = {}

dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
param_grid["DecisionTree"] = dt_param_grid

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4]
}
param_grid["RandomForest"] = rf_param_grid

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [6, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2]
}
param_grid["XGBoost"] = xgb_param_grid

lgbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [-1, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2]
}
param_grid["LGBM"] = lgbm_param_grid

catboost_param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}
param_grid["CatBoost"] = catboost_param_grid

In [ ]:
for size in sizes:
    X_training = X_train_preprocessed[:size]
    y_training = y_train[:size]

    for modelName in dtModels:
        start_time = time.time()

        # GridSearchCV with cross-validation for each model
        grid_search = GridSearchCV(dtModels[modelName], param_grid[modelName], cv=5, scoring='accuracy')
        grid_search.fit(X_training, y_training)

        # Print the best parameters
        print(f"Best Parameters for {modelName}: {grid_search.best_params_}")
        
        # Using the best estimator found by the grid search
        best_model = grid_search.best_estimator_
        best_model.fit(X_training, y_training)
    
        predictionsTrain = best_model.predict(X_training)
        #predictionsVal = best_model.predict(X_val)
        predictionsTest = best_model.predict(X_test_preprocessed)
        
        results[size]["Train"][modelName] = accuracy_score(y_training, predictionsTrain)
        #results[size]["Validation"][modelName] = accuracy_score(y_val, predictionsVal)
        results[size]["Test"][modelName] = accuracy_score(y_test, predictionsTest)
        
        print(modelName)
        print("--- %s seconds ---" % (time.time() - start_time))
        print(classification_report(y_test, predictionsTest))
        #print(classification_report(Y_test, binarized_predictionsTest))

In [ ]:
results

In [ ]:
results.to_csv("resultAdult.csv")

## Neural Networks

In [ ]:
for size in sizes:
    X_training, X_val, y_training, y_val = train_test_split(X_train_preprocessed[:size], y_train[:size], test_size=0.2, random_state=42)

    # Combine X and Y datasets
    X_training_torch = torch.tensor(X_training, dtype=torch.float32)
    y_training_torch = torch.tensor(y_training.values, dtype=torch.float32)
    X_val_torch = torch.tensor(X_val, dtype=torch.float32)
    y_val_torch = torch.tensor(y_val.values, dtype=torch.float32)
    
    # Assuming you have your dataset loaded as tensors: train_x, train_y, val_x, val_y
    train_dataset = TensorDataset(X_training_torch, y_training_torch)
    val_dataset = TensorDataset(X_val_torch, y_val_torch)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, persistent_workers=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=8, persistent_workers=True)

    for modelName in neuralNetsModels:
        print(modelName)
        start_time = time.time()

        # Define early stopping callback
        early_stopping_callback = EarlyStopping(
            monitor='val_acc',  # Monitor validation loss
            min_delta=0.001,     # Minimum change in validation loss to qualify as an improvement
            patience=20,          # Number of epochs with no improvement after which training will be stopped
            verbose=True,        # Print messages about early stopping
            mode='max'           # Minimize validation loss
        )

        # Define ModelCheckpoint callback to save the best model
        checkpoint_callback = ModelCheckpoint(
            dirpath='checkpoints',
            filename='modelName+str(size)',
            monitor='val_acc',
            mode='max',
            save_top_k=1
        )
        
        model_cloned = copy.deepcopy(neuralNetsModels[modelName])
        
        # Define TensorBoard logger
        logger = TensorBoardLogger('logs', name=modelName+str(size))

        # Create PyTorch Lightning trainer with TensorBoard logger
        trainer = pl.Trainer(accelerator="gpu", max_epochs=300, logger=logger, callbacks=[early_stopping_callback, checkpoint_callback])

        # Train the model
        trainer.fit(model_cloned, train_dataloader, val_dataloader)
        
        results[size]["Train"][modelName] = evaluate_binary_classification(model_cloned, train_dataloader)
        #results[size]["Validation"][modelName] = evaluate_binary_classification(model_cloned, val_dataloader)
        results[size]["Test"][modelName] = evaluate_binary_classification(model_cloned, test_dataloader, test = True)

        print("--- %s seconds ---" % (time.time() - start_time))
        #print(classification_report(Y_test, binarized_predictionsTest))

In [ ]:
results

# Neural Networks with embeddings

In [96]:
embeddings = {}

for categorical_feature in categorical_features:
    embeddings["cat__" + categorical_feature] = len(np.unique(X_train[categorical_feature]))

embeddings

{'cat__workclass': 9,
 'cat__education': 16,
 'cat__marital-status': 7,
 'cat__occupation': 15,
 'cat__relationship': 6,
 'cat__race': 5,
 'cat__gender': 2,
 'cat__native-country': 42}

In [308]:
tiny_model = EmbeddingModel(ModelClass = TinyModel, input_size = len(numerical_features), embeddings = embeddings)
small_model = EmbeddingModel(ModelClass = SmallModel, input_size = len(numerical_features), embeddings = embeddings)
medium_model = EmbeddingModel(ModelClass = MediumModel, input_size = len(numerical_features), embeddings = embeddings)
large_model = EmbeddingModel(ModelClass = LargeModel, input_size = len(numerical_features), embeddings = embeddings)
dropout_model = EmbeddingModel(ModelClass = DropoutModel, input_size = len(numerical_features), embeddings = embeddings)

neuralNetsModelsEmb = {
    "tiny_model_emb":tiny_model,
    "small_modelv": small_model,
    "medium_model_emb": medium_model,
    "large_model_emb": large_model,
    "dropout_model_emb": dropout_model
}

In [292]:
embeddingDataset = EmbeddingDataset(X_train, y_train)

In [309]:
for size in sizes:
    X_training, X_val, y_training, y_val = train_test_split(X_train[:size], y_train[:size], test_size=0.2, random_state=42)

    # Combine X and Y datasets
    embeddingDatasetTrain = EmbeddingDataset(X_training, y_training, preprocessor = embeddingDataset.preprocessor)
    embeddingDatasetVal = EmbeddingDataset(X_val, y_val, preprocessor = embeddingDatasetTrain.preprocessor)
    embeddingDatasetTest = EmbeddingDataset(X_test, y_test, preprocessor = embeddingDatasetTrain.preprocessor)
    
    embeddingDataloaderTrain = DataLoader(embeddingDatasetTrain, 
           batch_size=batch_size,
           batch_sampler=None, 
           num_workers=0, 
           collate_fn=embedding_collate_fn,
           pin_memory=False,
           worker_init_fn=None,
           persistent_workers=False
          )
    embeddingDataloaderVal = DataLoader(embeddingDatasetVal, 
           batch_size=batch_size,
           batch_sampler=None, 
           num_workers=0, 
           collate_fn=embedding_collate_fn,
           pin_memory=False,
           worker_init_fn=None,
           persistent_workers=False
          )
    embeddingDataloaderTest = DataLoader(embeddingDatasetTest, 
               batch_size=batch_size,
               batch_sampler=None, 
               num_workers=0, 
               collate_fn=embedding_collate_fn,
               pin_memory=False,
               worker_init_fn=None,
               persistent_workers=False
              )

    for modelName in neuralNetsModelsEmb:
        print(modelName)
        start_time = time.time()

        # Define early stopping callback
        early_stopping_callback = EarlyStopping(
            monitor='val_acc',  # Monitor validation loss
            min_delta=0.001,     # Minimum change in validation loss to qualify as an improvement
            patience=20,          # Number of epochs with no improvement after which training will be stopped
            verbose=True,        # Print messages about early stopping
            mode='max'           # Minimize validation loss
        )

        # Define ModelCheckpoint callback to save the best model
        checkpoint_callback = ModelCheckpoint(
            dirpath='checkpoints',
            filename=modelName+str(size),
            monitor='val_acc',
            mode='max',
            save_top_k=1
        )
        
        model_cloned = copy.deepcopy(neuralNetsModelsEmb[modelName])
        
        # Define TensorBoard logger
        logger = TensorBoardLogger('logs', name=modelName+str(size))

        # Create PyTorch Lightning trainer with TensorBoard logger
        trainer = pl.Trainer(accelerator="gpu", max_epochs=300, logger=logger, callbacks=[early_stopping_callback, checkpoint_callback])

        # Train the model
        trainer.fit(model_cloned, embeddingDataloaderTrain, embeddingDatasetVal)
        
        results[size]["Train"][modelName] = model_cloned.evaluate(embeddingDataloaderTrain)
        #results[size]["Validation"][modelName] = evaluate_binary_classification(model_cloned, val_dataloader)
        results[size]["Test"][modelName] = model_cloned.evaluate(embeddingDataloaderTest, test = True)

        print("--- %s seconds ---" % (time.time() - start_time))
        #print(classification_report(Y_test, binarized_predictionsTest))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\loris\anaconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:653: Checkpoint directory C:\Users\loris\Documents\Dottorato\Esperimenti\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params
--------------------------------------------
0 | accuracy | BinaryAccuracy | 0     
1 | model    | TinyModel      | 641   
--------------------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)


tiny_model_emb


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\loris\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\loris\anaconda3\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                      | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Metric val_acc improved by 0.009 >= min_delta = 0.001. New best score: 0.818


Validation: |                                                                                    | 0/? [00:00<…

Metric val_acc improved by 0.007 >= min_delta = 0.001. New best score: 0.825
C:\Users\loris\anaconda3\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


KeyboardInterrupt: 

In [ ]:
results

In [196]:
results.to_csv("resultAdult.csv")